# Latent Semantic Indexing

Here, we apply the technique **Latent Semantic Indexing** to capture the similarity of words.

In [25]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

words_list = list()
with open('lsiWords.txt') as f:
    for line in f:
        words_list.append(line.strip())
words = pd.Series(words_list, name="words")
word_frequencies = pd.read_csv('lsiMatrix.txt', sep=' ', index_col=False,
                               header=None, names=words)
word_frequencies.T.head(20)

,0,1,2,3,4,5,6,7,8
000,2,0,4,0,0,0,0,0,0
100,0,0,2,0,0,0,0,0,0
1913,0,0,0,3,0,0,0,0,0
1977,0,0,0,0,0,0,2,0,0
2001,0,0,0,0,0,0,0,2,0
4,0,0,2,0,0,0,0,0,0
5,0,0,2,0,0,0,0,0,0
500,2,0,0,0,0,0,0,0,0
"""",0,27,13,23,19,48,56,35,22
(,4,2,3,7,4,0,5,8,4
